In [2]:
import trax

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [43]:
import textwrap
wrapper = textwrap.TextWrapper(width=70)
import numpy as np
from trax.fastmath import numpy as jnp
from trax import layers as tl

In [19]:
train_stream_fn = trax.data.TFDS(
    'cnn_dailymail',
    data_dir= 'data/',
    keys= ('article', 'highlights'),
    train= True
)

eval_stream_fn = trax.data.TFDS(
    'cnn_dailymail',
    data_dir= 'data/',
    keys= ('article', 'highlights'),
    train= False
)

In [20]:
def tokenize(input_str, EOS=1):
    inputs = next(trax.data.tokenize(
        iter([input_str]),
        vocab_file= 'summarize32k.subword.subwords',
        vocab_dir= r'/kaggle/input/nlp-coursera-deep-learningai/Natural-Language-Processing-Specialization-master/Natural Language Processing with Attention Models/Week 2/vocab_dir'
    ))
    return list(inputs) + [EOS]

def detokenize(integers):
    s = trax.data.detokenize(
        integers,
        vocab_file= 'summarize32k.subword.subwords',
        vocab_dir= r'/kaggle/input/nlp-coursera-deep-learningai/Natural-Language-Processing-Specialization-master/Natural Language Processing with Attention Models/Week 2/vocab_dir'
    )
    return wrapper.fill(s)

In [21]:
EOS = 1
PAD = 0

def preprocess(stream):
    for (article, summary) in stream:
        joint = np.array(list(article) + [EOS, PAD] + list(summary) + [EOS])
        mask = [0] * (len(list(article)) + 2) + [1] * (len(list(summary)) + 1)
        yield joint, joint, np.array(mask)

input_pipeline = trax.data.Serial(
    trax.data.Tokenize(
        vocab_file= 'summarize32k.subword.subwords',
        vocab_dir= r'/kaggle/input/nlp-coursera-deep-learningai/Natural-Language-Processing-Specialization-master/Natural Language Processing with Attention Models/Week 2/vocab_dir'
    ),
    preprocess,
    trax.data.FilterByLength(2048)
)

train_stream = input_pipeline(train_stream_fn())
eval_stream = input_pipeline(eval_stream_fn())

train_input, train_target, train_mask = next(train_stream)

assert sum(train_input - train_target) == 0

In [22]:
print('Single example mask:')
print(train_mask)

print()

print('Single example:')
print(train_input)

print()

print('Single example: (detokenized)')
print(detokenize(train_input))

Single example mask:
[0 0 0 ... 1 1 1]

Single example:
[ 1208 13859    25 ...  4720    10     1]

Single example: (detokenized)
Royal Mail were blasted today after they banned deliveries to more
than 100 residents on a street following health and safety fears -
over wonky paving slabs. Residents on the Victorian estate in Stoke-
on-Trent, Staffordshire, are now forced to make a four-mile round trip
collect their letters and parcels from a central depot. Royal Mail
postmen are refusing to deliver to The Villas after it became a no-go
zone due to 'a number of incidents involving colleagues falling and
slipping'. Resident Catherine Burgass said the move by Royal Mail has
'annoyed' all of the residents at The Villas. Yesterday home-owners on
the road, which is not council-maintained as it is privately run by
the residents' association, said their mail had been delivered without
a problem for 150 years and slammed Royal Mail for not giving them any
warning. The estate's residents' associat

In [28]:
#Bucketing
boundaries =  [128, 256,  512, 1024]
batch_sizes = [16,    8,    4,    2, 1]

train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes)(train_stream)
eval_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes)(eval_stream)

input_batch, _, mask_batch = next(train_batch_stream)

print(input_batch.shape)
print(input_batch[0])

(1, 1054)
[ 1096  6318 11454 ...  5933    10     1]


In [29]:
print(detokenize(input_batch[0]))

Michael Higdon came off the bench and scored two late goals to send
Nigel Clough’s Sheffield United into the quarter-finals. Striker
Higdon, 31, headed his first on 86 minutes from Jose Baxter’s cross
and added his second four minutes later with a shot into the bottom
left-hand corner to snatch a dramatic victory. MK Dons’ two brightest
young prospects, both of interest to Premier League clubs, had
combined to put their side ahead. Michael Higdon (left) of Sheffield
United celebrates scoring the winning goal at Stadium mk. Higdon
strikes to score his second goal that sealed a Sheffield United
comeback. MK Dons (4-2-3-1): Martin 6; Spence 6, Flanagan 6, McFadzean
6, Lewington 6.5; Potter 6, Alli 7.5; Bowditch 6 (Powell 61min, 6),
Reeves 6 (Grigg 61, 6), Carruthers 6; Afobe 7. Subs not used:
McLoughlin, Kay, Randall, Baker, Hitchcock. Booked: Potter, Grigg.
Scorer: Afobe pen 67. Sheffield United (4-2-3-1): Howard 5.5; Alcock
4.5, McEveley, McGahey 6, Harris 5; Doyle 6, Reed 6; Davis 6
(C

In [36]:
#helper functions
def create_tensor(t):
    return jnp.array(t)

def display_tensor(t, name):
    print(f'{name} shape:', t.shape)
    print(t)
    print()
    
#initializing some random q, k, v and m
q = create_tensor([[1, 0, 0], [0, 1, 0]])
display_tensor(q, 'query')
k = create_tensor([[1, 2, 3], [4, 5, 6]])
display_tensor(k, 'key')
v = create_tensor([[0, 1, 0], [1, 0, 1]])
display_tensor(v, 'value')
m = create_tensor([[0, -1e9], [0, 0]])
display_tensor(m, 'mask')

query shape: (2, 3)
[[1 0 0]
 [0 1 0]]

key shape: (2, 3)
[[1 2 3]
 [4 5 6]]

value shape: (2, 3)
[[0 1 0]
 [1 0 1]]

mask shape: (2, 2)
[[ 0.e+00 -1.e+09]
 [ 0.e+00  0.e+00]]



In [49]:
#implementing the scaled dot-product attention
def DotProductAttention(query, key, value, mask):
    assert query.shape[-1] == key.shape[-1] == value.shape[-1]
    depth = query.shape[-1]
    dots = jnp.matmul(query, jnp.swapaxes(key, -1, -2)) / jnp.sqrt(depth)
    if mask is not None:
        dots = jnp.where(mask, dots, jnp.full_like(dots, -1e9))
    logsumexp = trax.fastmath.logsumexp(dots, axis=-1, keepdims=True)
    dots = jnp.exp(dots - logsumexp)
    attention = jnp.matmul(dots, value)
    return attention

def compute_attention_heads_closure(n_heads, d_head):
    #we are defining a closure to give freedom of choosing different n_heads, d_head values
    def compute_attention_head(x):
        batch_size = x.shape[0]
        seqlen = x.shape[1]
        x = jnp.reshape(x, (batch_size, seqlen, n_heads, d_head))
        x = jnp.transpose(x, (0, 2, 1, 3))
        x = jnp.reshape(x, (batch_size * n_heads, seqlen, d_head))
        return x
    return compute_attention_head

def dot_product_self_attention(q, k, v):
    mask_size = q.shape[1]
    mask = jnp.tril(jnp.ones((1, mask_size, mask_size)), k=0)
    return DotProductAttention(q, k, v, mask)

def compute_attention_output_closure(n_heads, d_head):
    def compute_attention_output(x):
        seqlen = x.shape[1]
        x = jnp.reshape(x, (-1, n_heads, seqlen, d_head))
        x = jnp.transpose(x, (0, 2, 1, 3))
        x = jnp.reshape(x, (-1, seqlen, n_heads * d_head))
        return x
    return compute_attention_output

def CausalAttention(
    d_feature,
    n_heads,
    compute_attention_heads_closure=compute_attention_heads_closure,
    dot_product_self_attention=dot_product_self_attention,
    compute_attention_output_closure=compute_attention_output_closure,
    mode='train'
):
    assert d_feature % n_heads == 0
    d_head = d_feature // n_heads
    ComputeAttentionHeads = tl.Fn('AttnHeads', compute_attention_heads_closure(n_heads, d_head))
    return tl.Serial(
        tl.Branch(
            [tl.Dense(n_units=d_feature), ComputeAttentionHeads],
            [tl.Dense(n_units=d_feature), ComputeAttentionHeads],
            [tl.Dense(n_units=d_feature), ComputeAttentionHeads]
        ),
        tl.Fn('DotProductAttn', dot_product_self_attention, n_out=1),
        tl.Fn('AttnOutput', compute_attention_output_closure(n_heads, d_head), n_out=1),
        tl.Dense(n_units=d_feature)
    )

In [50]:
print(CausalAttention(d_feature=512, n_heads=8))

Serial[
  Branch_out3[
    [Dense_512, AttnHeads]
    [Dense_512, AttnHeads]
    [Dense_512, AttnHeads]
  ]
  DotProductAttn_in3
  AttnOutput
  Dense_512
]


In [53]:
#Transformer Decoder Block
def DecoderBlock(d_model, d_ff, n_heads, dropout, mode, ff_activation):
    causal_attention = CausalAttention(d_model, n_heads)
    feed_forward = [
        tl.LayerNorm(),
        tl.Dense(n_units=d_ff),
        ff_activation(),
        tl.Dropout(rate=dropout, mode=mode),
        tl.Dense(n_units=d_model),
        tl.Dropout(rate=dropout, mode=mode)
    ]
    return tl.Serial(
        tl.Residual(
            tl.LayerNorm(),
            causal_attention,
            tl.Dropout(rate=dropout, mode=mode)
        ),
        tl.Residual(
            feed_forward
        )
    )

# Take a look at the decoder block
print(DecoderBlock(d_model=512, d_ff=2048, n_heads=8, dropout=0.1, mode='train', ff_activation=tl.Relu))

Serial[
  Serial[
    Branch_out2[
      None
      Serial[
        LayerNorm
        Serial[
          Branch_out3[
            [Dense_512, AttnHeads]
            [Dense_512, AttnHeads]
            [Dense_512, AttnHeads]
          ]
          DotProductAttn_in3
          AttnOutput
          Dense_512
        ]
        Dropout
      ]
    ]
    Add_in2
  ]
  Serial[
    Branch_out2[
      None
      Serial[
        LayerNorm
        Dense_2048
        Serial[
          Relu
        ]
        Dropout
        Dense_512
        Dropout
      ]
    ]
    Add_in2
  ]
]


In [56]:
def TransformerLM(vocab_size=33300, n_heads=8, d_model=512, d_ff=2048, dropout=0.1, max_len=4096, ff_activation=tl.Relu, n_layers=6, mode='train'):
    positional_encoder = [
        tl.Embedding(vocab_size=vocab_size, d_feature=d_model),
        tl.Dropout(rate=dropout, mode=mode),
        tl.PositionalEncoding(max_len=max_len, mode=mode)
    ]
    decoder_blocks = [
        DecoderBlock(d_model, d_ff, n_heads, dropout, mode, ff_activation) for _ in range(n_layers)
    ]
    return tl.Serial(
        tl.ShiftRight(mode=mode),
        positional_encoder,
        decoder_blocks,
        tl.LayerNorm(),
        tl.Dense(n_units=vocab_size),
        tl.LogSoftmax()
    )

# Take a look at the Transformer
print(TransformerLM(n_layers=1))

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_33300_512
  Dropout
  PositionalEncoding
  Serial[
    Serial[
      Branch_out2[
        None
        Serial[
          LayerNorm
          Serial[
            Branch_out3[
              [Dense_512, AttnHeads]
              [Dense_512, AttnHeads]
              [Dense_512, AttnHeads]
            ]
            DotProductAttn_in3
            AttnOutput
            Dense_512
          ]
          Dropout
        ]
      ]
      Add_in2
    ]
    Serial[
      Branch_out2[
        None
        Serial[
          LayerNorm
          Dense_2048
          Serial[
            Relu
          ]
          Dropout
          Dense_512
          Dropout
        ]
      ]
      Add_in2
    ]
  ]
  LayerNorm
  Dense_33300
  LogSoftmax
]


In [ ]:
#for training and evaluation --> refer actual lab file